In [ ]:
from flask import Flask, request, jsonify
import cx_Oracle
from datetime import datetime

app = Flask(__name__)

# Oracle database connection settings
db_user = 'cgi_24IS_IoT3_p2_4'
db_password = 'smhrd4'
db_host = 'project-db-cgi.smhrd.com'
db_port = '1524'
db_sid = 'xe'

# Create database connection
dsn_tns = cx_Oracle.makedsn(db_host, db_port, sid=db_sid)
connection = cx_Oracle.connect(user=db_user, password=db_password, dsn=dsn_tns)

# Sequence for primary key generation
SEQUENCE_NAME = 'sensor_idx_sequence'

def get_next_sequence_value():
    cursor = connection.cursor()
    cursor.execute(f'SELECT {SEQUENCE_NAME}.nextval FROM dual')
    result = cursor.fetchone()[0]
    cursor.close()
    return result

@app.route('/api/data', methods=['POST'])
def receive_data():
    try:
        data = request.json
        if not data:
            return jsonify({'error': 'No data provided'}), 400

        # Get server's current date and time
        current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Parse latitude and longitude directly from the request
        latitude = data['latitude']  # Received latitude
        longitude = data['longitude']  # Received longitude
        smoke_loc = f"{latitude}, {longitude}"  # Combine latitude and longitude in "latitude,longitude" format

        sensor_idx = get_next_sequence_value()
        with connection.cursor() as cursor:
            insert_query = """
            INSERT INTO tb_smoking_sensor (
                SENSOR_IDX, USER_EMAIL, SMOKE_TIME, SMOKE_LOC, 
                SENSOR1_VALUE, SENSOR2_VALUE, SENSOR3_VALUE, SMOKE_DATE
            ) VALUES (
                :1, :2, TO_TIMESTAMP(:3, 'YYYY-MM-DD HH24:MI:SS'), :4, :5, :6, :7, TO_DATE(:9, 'YYYY-MM-DD'))
            """
            cursor.execute(insert_query, (
                sensor_idx, data['user_email'], current_datetime, smoke_loc,
                data['sensor1_value'], data['sensor2_value'], 
                data['sensor3_value'], 
                current_datetime.split()[0]  # Use server's current date
            ))
            connection.commit()

        return jsonify({'message': 'Data received and stored successfully'}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=3000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3000
 * Running on http://192.168.219.69:3000
Press CTRL+C to quit
